# Test de scanpy sur notre dataset d'entrainement projet DIVIDE

Pour se connecter à ce notebook 
Besoin de lancer  l'image singularity de : _Containers/shortcakelight.sif

https://github.com/rnakato/ShortCake/tree/master
Est conçu spécifiquement pour les analyses de données single cell

Pour lancer l'image singularity et pouvoir se connecter aux notebooks jupyter:
srun --pty bash
singularity exec 6_Containers/shortcakelight.sif jupyternotebook.sh

puis faire le lien ssh sur l'ordi avec cette commande  que l'on ecrit dans le terminal local: 
ssh -A -t -t vgoupille@genossh.genouest.org -L 8888:localhost:8888 ssh cl1n030 -L 8888:localhost:8888
modifier le port (numero et noeud si besoin)

si probleme de connection 

ls -a
rm -r .vscode-server
puis se remet sur vscode 
et refait la liaison avec ssh remote (et reinstaller les extensions...)


# Définir les chemins des fichiers
ch_barcodes = "/home/genouest/cnrs_umr6553/vgoupille/DOL_scRNAseq/4_results/1_script_results/1_results_STARsolo/starsolo_output/Solo.out/GeneFull/filtered/barcodes.tsv"
ch_features = "/home/genouest/cnrs_umr6553/vgoupille/DOL_scRNAseq/4_results/1_script_results/1_results_STARsolo/starsolo_output/Solo.out/GeneFull/filtered/features.tsv"

ch_matrix_file = "/home/genouest/cnrs_umr6553/vgoupille/DOL_scRNAseq/4_results/1_script_results/1_results_STARsolo/starsolo_output/Solo.out/GeneFull/filtered/matrix.mtx"

#ch_matrix_file = "/home/genouest/cnrs_umr6553/vgoupille/DOL_scRNAseq/4_results/1_script_results/1_results_STARsolo/starsolo_output/Solo.out/GeneFull/raw/UniqueAndMult-Uniform.mtx"

Ce code charge les données scRNA-seq issues de STARsolo dans un objet AnnData de Scanpy. Voici une explication détaillée :

📌 1. Définition du chemin des fichiers STARsolo

data_dir = '/home/genouest/cnrs_umr6553/vgoupille/DOL_scRNAseq/4_results/1_script_results/1_results_STARsolo/starsolo_output/Solo.out/GeneFull/filtered/'

	•	Définit le chemin du dossier contenant les fichiers matrix.mtx, features.tsv, et barcodes.tsv.

📌 2. Chargement de la matrice d’expression des gènes

adata = sc.read(data_dir + 'matrix.mtx', cache=True).T  # transpose the data

	•	sc.read(data_dir + 'matrix.mtx', cache=True): Charge la matrice d’expression stockée dans matrix.mtx (format Matrix Market).
	•	.T: Transpose la matrice car matrix.mtx est souvent stockée sous forme (gènes x cellules), alors que Scanpy attend une forme (cellules x gènes).

📌 3. Attribution des noms des gènes

adata.var_names = pd.read_csv(data_dir + 'features.tsv', header=None, sep='\t')[1]

	•	pd.read_csv(..., sep='\t') : Charge le fichier features.tsv, qui contient les noms des gènes.
	•	[1] : Sélectionne la deuxième colonne (généralement les noms de gènes plutôt que les IDs).
	•	adata.var_names = ... : Associe ces noms aux colonnes de la matrice (les gènes).

📌 4. Attribution des noms des cellules

adata.obs_names = pd.read_csv(data_dir + 'barcodes.tsv', header=None)[0]

	•	pd.read_csv(...)[0] : Charge la première colonne du fichier barcodes.tsv, qui contient les noms des cellules.
	•	adata.obs_names = ... : Associe ces noms aux lignes de la matrice (les cellules).

📌 5. Conversion en format sparse (optimisation mémoire)

adata.X = csr_matrix(adata.X)  # enforce the CSR format

	•	adata.X stocke la matrice d’expression.
	•	csr_matrix(adata.X): Convertit adata.X en Compressed Sparse Row format (CSR), un format optimisé pour réduire l’utilisation mémoire et accélérer les calculs.

📌 6. Affichage des informations sur l’objet AnnData

print(adata)

	•	Affiche un résumé de l’objet AnnData, contenant :
	•	Le nombre de cellules (n_obs).
	•	Le nombre de gènes (n_vars).
	•	Le format des données (adata.X).

✅ Résumé du rôle du code
	1.	Charge la matrice de comptage issue de STARsolo.
	2.	Ajoute les noms des gènes et noms des cellules.
	3.	Convertit les données en format sparse pour économiser de la mémoire.
	4.	Stocke le tout dans un objet AnnData pour analyse avec Scanpy.

🔍 Problèmes potentiels
	2.	Scanpy propose read_10x_mtx() qui simplifie cette étape
→ À essayer pour charger directement matrix.mtx et ses métadonnées.



In [67]:
# Load libraries and abreviate them
import pandas as pd
import scanpy as sc
import numpy as np
from scipy.sparse import csr_matrix
import pprint as pp


# Définition du chemin des fichiers
data_dir = "/home/genouest/cnrs_umr6553/vgoupille/DOL_scRNAseq/4_results/1_script_results/1_results_STARsolo/starsolo_output/Solo.out/GeneFull/filtered/"

# Chargement des barcodes et des gènes
barcodes = pd.read_csv(f"{data_dir}barcodes.tsv", header=None, sep="\t")[0]
features = pd.read_csv(f"{data_dir}features.tsv", header=None, sep="\t")

# Chargement de la matrice en format sparse
adata = sc.read_mtx(f"{data_dir}matrix.mtx").T  # Transposition pour correspondre aux dimensions attendues

# Attribution des noms aux cellules (obs) et aux gènes (var)
adata.obs_names = barcodes
adata.var_names = features[1]  # Deuxième colonne pour les noms de gènes

# Ajout des métadonnées minimales
adata.obs["barcode"] = adata.obs.index
adata.var["gene_id"] = adata.var.index

# Conversion explicite en format sparse pour optimiser la mémoire
adata.X = csr_matrix(adata.X)

# Si tu veux sauvegarder les données brutes dans adata.raw, tu peux faire cela juste après le chargement des données :
adata.raw = adata  # Sauvegarde l'objet complet dans adata.raw

# Récupérer les données brutes avant filtrage
#adata_raw = adata.raw.to_adata()  
#adata_raw.X = adata_raw.X.toarray()  # S'assurer que les données sont dans un format dense

# Affichage des informations principales
print(adata)  
#print(adata.obs.head())  # Vérification des cellules
#print(adata.var.head())  # Vérification des gènes

pp.pprint(adata.obs.head())  # Vérification des cellules
pp.pprint(adata.var.head())  # Vérification des gènes

AnnData object with n_obs × n_vars = 16 × 6248
    obs: 'barcode'
    var: 'gene_id'
                                               barcode
AAACATCG_AAACATCG_AAACGATA  AAACATCG_AAACATCG_AAACGATA
AACGTGAT_AAACATCG_AAACGATA  AACGTGAT_AAACATCG_AAACGATA
AAACATCG_AACGTGAT_AAACGATA  AAACATCG_AACGTGAT_AAACGATA
AACGTGAT_AACGTGAT_AAACGATA  AACGTGAT_AACGTGAT_AAACGATA
AAACATCG_AAACATCG_ACTCGTAA  AAACATCG_AAACATCG_ACTCGTAA
                 gene_id
QLH64_00005  QLH64_00005
QLH64_00010  QLH64_00010
QLH64_00015  QLH64_00015
QLH64_00020  QLH64_00020
rrf                  rrf


In [ ]:
#print(adata.raw)  # Vérification des données brutes

Raw AnnData with n_obs × n_vars = 16 × 6248
    var: 'gene_id'
<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 1856 stored elements and shape (16, 6248)>
  Coords	Values
  (0, 3)	1.0
  (0, 101)	1.0
  (0, 4560)	1.0
  (1, 3)	2.0
  (1, 192)	1.0
  (1, 4067)	1.0
  (2, 3)	1.0
  (2, 101)	1.0
  (2, 901)	1.0
  (2, 3482)	1.0
  (2, 4535)	1.0
  (2, 4554)	1.0
  (3, 3643)	1.0
  (4, 0)	1.0
  (4, 3)	4.0
  (4, 101)	10.0
  (4, 910)	1.0
  (4, 958)	3.0
  (4, 2258)	2.0
  (4, 2940)	1.0
  (4, 3814)	2.0
  (4, 4043)	2.0
  (4, 4072)	3.0
  (4, 4540)	1.0
  (4, 4545)	36.0
  :	:
  (15, 5955)	1.0
  (15, 5962)	18.0
  (15, 5963)	134.0
  (15, 5964)	196.0
  (15, 5965)	9.0
  (15, 5966)	2.0
  (15, 5967)	1.0
  (15, 5968)	8.0
  (15, 5972)	6.0
  (15, 5977)	8.0
  (15, 6009)	86.0
  (15, 6012)	33.0
  (15, 6032)	1.0
  (15, 6099)	20.0
  (15, 6156)	21.0
  (15, 6160)	16.0
  (15, 6161)	10.0
  (15, 6162)	55.0
  (15, 6173)	65.0
  (15, 6175)	4.0
  (15, 6176)	2.0
  (15, 6189)	24.0
  (15, 6204)	2.0
  (15, 6207)	62.0
  (15, 

In [ ]:
import os
#print(os.getcwd())  # Affiche le répertoire de travail actuel
os.makedirs("4_results/1_script_results/3_resuts_scanpy", exist_ok=True)  # Crée le dossier s'il n'existe pas
adata.write("4_results/1_script_results/3_resuts_scanpy/data.h5ad")  # Sauvegarde dans le dossier "results"
print(os.path.exists("4_results/1_script_results/3_resuts_scanpy/data.h5ad"))  # True si le fichier est bien enregistré

In [ ]:
# Charger les données
adata = sc.read("4_results/1_script_results/3_resuts_scanpy/data.h5ad")
# Afficher les dimensions
pp.pprint(adata)  # Affiche les dimensions de la matrice d'expression

# Afficher les 16 premières cellules et les 10 premiers gènes
pp.pprint(adata[:16, :10].to_df())


In [ ]:
# verifier si pas de doublons de nom dans les cellules 
print(adata.obs.index.duplicated().sum())
# verifier si pas de doublons dans les gènes
print(adata.var.index.duplicated().sum())
# 132 doublons dans les nom de gènes
# print les doublons
print(adata.var.index[adata.var.index.duplicated()])

pp.pprint(adata.var.index[adata.var.index.duplicated()].tolist())

In [ ]:
adata.obs["n_genes_by_cells"] = (adata.X > 0).sum(axis=1)  # Nombre de gènes détectés par cellule
print(adata.obs["n_genes_by_cells"])

adata.obs["total_counts"] = adata.X.sum(axis=1).A1  # Nombre total de comptes (comptages) par cellule

# Make variable names unique
adata.var_names_make_unique()

# violon plot
sc.pl.violin(
    adata,
    ["n_genes_by_cells", "total_counts"],
    jitter=0.4,
    size=8,
    multi_panel=True,
)

In [ ]:
# Vérifier la forme de la matrice de données (nombre de cellules et de gènes)
n_cells, n_genes = adata.shape
print(f"Nombre de cellules: {n_cells}, Nombre de gènes: {n_genes}")



In [ ]:
sc.pp.filter_cells(adata, min_genes=200)  # Garde cellules avec au moins 200 gènes => reste uniquement 4 cellules
sc.pp.filter_genes(adata, min_cells=4)  # Garde gènes exprimés dans au moins 4  cellules ::: donc ici l'ensembles 4 cellules restantes 

# Vérifier la forme de la matrice de données (nombre de cellules et de gènes)
n_cells, n_genes = adata.shape
print(f"Nombre de cellules: {n_cells}, Nombre de gènes: {n_genes}")


In [ ]:
# Récupérer les données brutes avant filtrage
adata_raw = adata.raw.to_adata()  
adata_raw.X = adata_raw.X.toarray()  # S'assurer que les données sont dans un format dense

# Vérification que les conditions existent dans adata.obs
print(adata_raw.obs['condition'].unique())

# Agréger les données par condition en utilisant pandas
# On suppose ici que 'condition' est une colonne dans adata.obs qui représente les différentes conditions.
adata_bulk = adata_raw.copy()  # Créer une copie pour l'agrégation

# Utilisation de pandas pour regrouper les données par condition
# adata_raw.obs['condition'] contient les conditions expérimentales associées aux cellules
grouped_data = pd.DataFrame(adata_raw.X, index=adata_raw.obs_names)
grouped_data['condition'] = adata_raw.obs['condition'].values

# Agrégée par 'condition' : somme des comptes d'expressions géniques pour chaque condition
adata_bulk.X = grouped_data.groupby('condition').sum().values

# Mettre à jour les noms des cellules dans adata_bulk
adata_bulk.obs_names = grouped_data['condition'].unique()

# Afficher les résultats
print(adata_bulk)